<a href="https://colab.research.google.com/github/piotrlnordea/Flan/blob/main/Kopia_notatnika_SpeechT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SpeechT5 with Hugging Face

Also check out the blog post: [hf.co/blog/speecht5](http://hf.co/blog/speecht5)

And the online demos:

- [Speech Synthesis (TTS)](https://huggingface.co/spaces/Matthijs/speecht5-tts-demo)
- [Voice Conversion](https://huggingface.co/spaces/Matthijs/speecht5-vc-demo)
- [Automatic Speech Recognition](https://huggingface.co/spaces/Matthijs/speecht5-asr-demo)

First install Transformers and sentencepiece. 

**Note:** It's important to restart the notebook after installing sentencepiece, or the demos won't work!

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hjxkly20
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hjxkly20
  Resolved https://github.com/huggingface/transformers.git to commit fe616f35c8d4355addb600ff58861bf63a921ccf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6419541 sha256=0fe96337ffc16440eb0eabf8144cd3a56bee43831e62875126e8c8f69a2529c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-83cec5b3/wheels/05/0a/97/64ae47c27ba95fae2cb5838e7b4b724

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00


## Text-to-speech

Load the model:

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

Preprocess the text input:

In [ ]:
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt")

Load a speaker embedding:

In [ ]:
from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

import torch
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


In [ ]:
speaker_embeddings.shape

torch.Size([1, 512])

Load a vocoder:

In [ ]:
from transformers import SpeechT5HifiGan
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

Generate the speech from the input text:

In [ ]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

AttributeError: ignored

In [ ]:
speech.shape

torch.Size([31232])

In [ ]:
from IPython.display import Audio

Audio(speech, rate=16000)

In [ ]:
import soundfile as sf
sf.write("tts_example.wav", speech.numpy(), samplerate=16000)

## Speech-to-speech for voice conversion

Load the model:

In [ ]:
from transformers import SpeechT5ForSpeechToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")
model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")

Load an input speech example:

In [ ]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
example = dataset[40]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset librispeech_asr_demo downloaded and prepared to /root/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b. Subsequent calls will reuse this data.


In [ ]:
Audio(example["audio"]["array"], rate=16000)

Preprocess the speech input:

In [ ]:
sampling_rate = dataset.features["audio"].sampling_rate
inputs = processor(audio=example["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

Load the speaker embedding for the target speaker's voice:

In [ ]:
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generate the speech:

In [ ]:
speech = model.generate_speech(inputs["input_values"], speaker_embeddings, vocoder=vocoder)

In [ ]:
Audio(speech, rate=16000)

In [ ]:
import soundfile as sf
sf.write("speech_converted.wav", speech.numpy(), samplerate=16000)

## Automatic speech recognition (using pipeline)

In [ ]:
from transformers import pipeline
generator = pipeline(task="automatic-speech-recognition", model="microsoft/speecht5_asr")

In [ ]:
transcription = generator(example["audio"]["array"])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1268: UserWarning: Using `max_length`'s default (450) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
transcription["text"]

'a man said to the universe sir i exist'

## Automatic speech recognition (using the model)

Load the model:

In [ ]:
from transformers import SpeechT5ForSpeechToText

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_asr")
model = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr")

Preprocess the input speech example:

In [ ]:
sampling_rate = dataset.features["audio"].sampling_rate
inputs = processor(audio=example["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

Generate text from the speech input:

In [ ]:
predicted_ids = model.generate(**inputs, max_length=100)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

In [ ]:
transcription[0]

'a man said to the universe sir i exist'